In [4]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [5]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'spa.txt'

In [6]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, 'r', encoding='utf-8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 86
Max sequence length for inputs: 17
Max sequence length for outputs: 42


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [164]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Very Dangerous Training Below

In [14]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 43s 5ms/step - loss: 1.1582 - val_loss: 1.2172
Epoch 2/100
8000/8000 [==============================] - 44s 6ms/step - loss: 0.9516 - val_loss: 1.0279
Epoch 3/100
8000/8000 [==============================] - 46s 6ms/step - loss: 0.8317 - val_loss: 0.9586
Epoch 4/100
8000/8000 [==============================] - 46s 6ms/step - loss: 0.7701 - val_loss: 0.9019
Epoch 5/100
8000/8000 [==============================] - 44s 6ms/step - loss: 0.7242 - val_loss: 0.8475
Epoch 6/100
8000/8000 [==============================] - 45s 6ms/step - loss: 0.6848 - val_loss: 0.8213
Epoch 7/100
8000/8000 [==============================] - 44s 6ms/step - loss: 0.6507 - val_loss: 0.7868
Epoch 8/100
8000/8000 [==============================] - 45s 6ms/step - loss: 0.6202 - val_loss: 0.7562
Epoch 9/100
8000/8000 [==============================] - 46s 6ms/step - loss: 0.5936 - val_loss: 0.7455
Epoch 10/100
800

8000/8000 [==============================] - 51s 6ms/step - loss: 0.0790 - val_loss: 1.0716
Epoch 79/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0772 - val_loss: 1.0837
Epoch 80/100
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0756 - val_loss: 1.0876
Epoch 81/100
8000/8000 [==============================] - 52s 6ms/step - loss: 0.0745 - val_loss: 1.0944
Epoch 82/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0725 - val_loss: 1.1057
Epoch 83/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0713 - val_loss: 1.1018
Epoch 84/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0695 - val_loss: 1.1223
Epoch 85/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0683 - val_loss: 1.1248
Epoch 86/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.0677 - val_loss: 1.1242
Epoch 87/100
8000/8000 [==============================] - 52s 6ms/st

C:\Users\migue\Anaconda3\lib\site-packages\keras\engine\topology.py:2361: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [15]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

In [16]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [17]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [289]:
for seq_index in range(1000):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Hi.
Decoded sentence: Hola.

-
Input sentence: Run!
Decoded sentence: ¡Corre!

-
Input sentence: Who?
Decoded sentence: ¿Quién?

-
Input sentence: Wow!
Decoded sentence: ¡Órale!

-
Input sentence: Fire!
Decoded sentence: ¡Fuego!

-
Input sentence: Fire!
Decoded sentence: ¡Fuego!

-
Input sentence: Fire!
Decoded sentence: ¡Fuego!

-
Input sentence: Help!
Decoded sentence: ¡Ayuda!

-
Input sentence: Help!
Decoded sentence: ¡Ayuda!

-
Input sentence: Help!
Decoded sentence: ¡Ayuda!

-
Input sentence: Jump!
Decoded sentence: ¡Salta!

-
Input sentence: Jump.
Decoded sentence: Salte.

-
Input sentence: Stop!
Decoded sentence: ¡Parad!

-
Input sentence: Stop!
Decoded sentence: ¡Parad!

-
Input sentence: Stop!
Decoded sentence: ¡Parad!

-
Input sentence: Wait!
Decoded sentence: ¡Espera!

-
Input sentence: Wait.
Decoded sentence: Esperen.

-
Input sentence: Keep it.
Decoded sentence: Guárdalo.

-
Input sentence: Me, too.
Decoded sentence: Yo, también.

-
Input sentence: Open up.
Decoded sentence: Abre.

-
Input sentence: Perfect!
Decoded sentence: ¡Perfecto!

-
Input sentence: See you.
Decoded sentence: Nos vemos.

-
Input sentence: Show me.
Decoded sentence: Muéstramelo.

-
Input sentence: Show me.
Decoded sentence: Muéstramelo.

-
Input sentence: Show me.
Decoded sentence: Muéstramelo.

-
Input sentence: Shut up!
Decoded sentence: ¡Cerrá el pico!

-
Input sentence: Shut up!
Decoded sentence: ¡Cerrá el pico!

-
Input sentence: Skip it.
Decoded sentence: Sáltatelo.

-
Input sentence: So long.
Decoded sentence: Hasta luego.

-
Input sentence: So long.
Decoded sentence: Hasta luego.

-
Input sentence: Stop it.
Decoded sentence: Deténlo.

-
Input sentence: Stop it.
Decoded sentence: Deténlo.

-
Input sentence: Take it.
Decoded sentence: Cógelo.

-
Input sentence: Tell me.
Decoded sentence: Decime.

-
Input sentence: Tom ate

-
Input sentence: I'm cool.
Decoded sentence: Estoy tranquilo.

-
Input sentence: I'm done.
Decoded sentence: Me pararne.

-
Input sentence: I'm easy.
Decoded sentence: Estoy mintiendo.

-
Input sentence: I'm fair.
Decoded sentence: Soy justo.

-
Input sentence: I'm fine.
Decoded sentence: Estoy enojorado.

-
Input sentence: I'm free!
Decoded sentence: ¡Soy libre!

-
Input sentence: I'm free.
Decoded sentence: Yo soy libre.

-
Input sentence: I'm full.
Decoded sentence: Estoy llena.

-
Input sentence: I'm full.
Decoded sentence: Estoy llena.

-
Input sentence: I'm full.
Decoded sentence: Estoy llena.

-
Input sentence: I'm here.
Decoded sentence: Estoy aquí.

-
Input sentence: I'm home.
Decoded sentence: Estoy en casa.

-
Input sentence: I'm hurt.
Decoded sentence: Estoy herido.

-
Input sentence: I'm late.
Decoded sentence: Llego tarde.

-
Input sentence: I'm lazy.
Decoded sentence: Soy vago.

-
Input sentence: I'm lost.
Decoded sentence: Estoy perdida.

-
Input sentence: I'm mean.
De

-
Input sentence: Catch him.
Decoded sentence: Atrápalo.

-
Input sentence: Come back.
Decoded sentence: Vuelve.

-
Input sentence: Come here.
Decoded sentence: Venig aquí.

-
Input sentence: Come here.
Decoded sentence: Venig aquí.

-
Input sentence: Come here.
Decoded sentence: Venig aquí.

-
Input sentence: Come home.
Decoded sentence: Ven a casa.

-
Input sentence: Come soon.
Decoded sentence: Vuelve a casa.

-
Input sentence: Do it now.
Decoded sentence: Hazlo ahora.

-
Input sentence: Dogs bark.
Decoded sentence: Consalo.

-
Input sentence: Don't ask.
Decoded sentence: No preguntes.

-
Input sentence: Don't cry.
Decoded sentence: No lloren.

-
Input sentence: Don't cry.
Decoded sentence: No lloren.

-
Input sentence: Don't cry.
Decoded sentence: No lloren.

-
Input sentence: Don't cry.
Decoded sentence: No lloren.

-
Input sentence: Don't lie.
Decoded sentence: No mintáis.

-
Input sentence: Fantastic!
Decoded sentence: ¡Fantástico!

-
Input sentence: Fantastic!
Decoded sentence:

-
Input sentence: It stinks.
Decoded sentence: Apesta.

-
Input sentence: It worked.
Decoded sentence: Funciona.

-
Input sentence: It's 3:10.
Decoded sentence: Son las tres y diez.

-
Input sentence: It's 8:30.
Decoded sentence: Son las 8:30.

-
Input sentence: It's a TV.
Decoded sentence: Es un televisor.

-
Input sentence: It's cold.
Decoded sentence: Hace frío.

-
Input sentence: It's cold.
Decoded sentence: Hace frío.

-
Input sentence: It's cool.
Decoded sentence: Está fresco.

-
Input sentence: It's cool.
Decoded sentence: Está fresco.

-
Input sentence: It's dark.
Decoded sentence: Está oscura.

-
Input sentence: It's done!
Decoded sentence: ¡Está listo!

-
Input sentence: It's fine.
Decoded sentence: Está perfectam.

-
Input sentence: It's hers.
Decoded sentence: Es suyo.

-
Input sentence: It's late.
Decoded sentence: Es taba.

-
Input sentence: It's mine.
Decoded sentence: Es mía.

-
Input sentence: It's mine.
Decoded sentence: Es mía.

-
Input sentence: It's nice.
Decoded s

-
Input sentence: Tom bowed.
Decoded sentence: Tom gritó.

-
Input sentence: Tom cared.
Decoded sentence: Tom se preocupaba.

-
Input sentence: Tom cared.
Decoded sentence: Tom se preocupaba.

-
Input sentence: Tom cared.
Decoded sentence: Tom se preocupaba.

-
Input sentence: Tom cares.
Decoded sentence: Tom se preocupa.

-
Input sentence: Tom cares.
Decoded sentence: Tom se preocupa.

-
Input sentence: Tom cried.
Decoded sentence: Tom lloró.

-
Input sentence: Tom dozed.
Decoded sentence: Tom dormitó.

-
Input sentence: Tom drove.
Decoded sentence: Tomás condujo.

-
Input sentence: Tom is OK.
Decoded sentence: Tom está bien.

-
Input sentence: Tom is in.
Decoded sentence: Tom está dentro.

-
Input sentence: Tom is up.
Decoded sentence: Tom está levantado.

-
Input sentence: Tom knits.
Decoded sentence: Tom teje.

-
Input sentence: Tom knows.
Decoded sentence: Tom sabe.

-
Input sentence: Tom moved.
Decoded sentence: Tom se me llavó.

-
Input sentence: Tom moved.
Decoded sentence: Tom

-
Input sentence: Don't sing.
Decoded sentence: No cantes.

-
Input sentence: Don't stop.
Decoded sentence: No pares.

-
Input sentence: Don't talk!
Decoded sentence: ¡No os quedaces!

-
Input sentence: Don't talk.
Decoded sentence: No hables.

-
Input sentence: Don't wait.
Decoded sentence: ¡No esperes!

-
Input sentence: Don't wait.
Decoded sentence: ¡No esperes!

-
Input sentence: Don't wait.
Decoded sentence: ¡No esperes!

-
Input sentence: Don't yell.
Decoded sentence: No grites.

-
Input sentence: Eat slowly.
Decoded sentence: Come despacio.

-
Input sentence: Eat slowly.
Decoded sentence: Come despacio.

-
Input sentence: Fire burns.
Decoded sentence: El fuego quema.

-
Input sentence: Follow Tom.
Decoded sentence: Síguelo a Tomás.

-
Input sentence: Follow Tom.
Decoded sentence: Síguelo a Tomás.

-
Input sentence: Follow Tom.
Decoded sentence: Síguelo a Tomás.

-
Input sentence: Follow Tom.
Decoded sentence: Síguelo a Tomás.

-
Input sentence: Follow him.
Decoded sentence: Sígu

-
Input sentence: I screamed.
Decoded sentence: Grité.

-
Input sentence: I screamed.
Decoded sentence: Grité.

-
Input sentence: I shot Tom.
Decoded sentence: Disparé a Tom.

-
Input sentence: I survived.
Decoded sentence: Sobreviví.

-
Input sentence: I think so.
Decoded sentence: Creo que sí.

-
Input sentence: I think so.
Decoded sentence: Creo que sí.

-
Input sentence: I threw up.
Decoded sentence: Comi cabo.



In [298]:
my_input_data = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

In [301]:
#sentence = "Get out"

for t, char in enumerate("Gracias y hola."):
    my_input_data[0, t, input_token_index[char]] = 1.

In [300]:

#decoded_sentence = decode_sequence(encoder_input_data[0:1])
decoded_sentence = decode_sequence(my_input_data)
print('-')
print('Input sentence:')
print('Decoded sentence:', decoded_sentence)

-
Input sentence:
Decoded sentence: igante un mito de encospardo.



In [159]:
encoder_input_data[0].shape

(17, 71)

In [160]:
encoder_input_data[0:1].shape

(1, 17, 71)

In [162]:
my_input_data.shape

(1, 17, 71)